In [3]:
import numpy as np

class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func) :
        self.creator = func
    
    def backward(self) :
        funcs = [self.creator]
        
        while funcs :
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator != None :
                funcs.append(x.creator)
            
        
class Function :
    def __call__(self, inputs) :
        xs = [x.data for x in inputs]
        ys = self.forward(xs) # 구체적인 계산은 forward에서 이루어짐
        outputs = [Variable(as_array(y.data)) for y in ys]
        # 출력값에 출처 표기
        for output in outputs :
            output.set_creator(self)
        
        # 함수 입출력 변수 표기
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs) :
        raise NotImplementedError()
    
    def backward(self, gys) :
        raise NotImplementedError()
        

class Square(Function) :
    def forward(self, x) :
        return x ** 2
    
    def backward(self, gy) :
        x = self.input.data
        gx = 2 * x * gy
        return gx

def square(x) :
    f = Square()
    return f(x)

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)
    
    def backward(self, gy) :
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x) :
    f = Exp()
    return f(x)

class Add(Function) :
    def forward(self, xs) :
        x0, x1 = xs
        y = x0 + x1
        return (y,)

def add(xs) :
    f = Add()
    return f(xs)

def numerical_diff(f, x, eps = 1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

def as_array(x) :
    if np.isscalar(x) :
        return np.array(x)
    return x

In [5]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
ys = add(xs)
y = ys[0]
print(y.data)

5


In [8]:
np.isscalar(5)

True